In [1]:
from transformers import pipeline
import keras
import numpy as np
import pickle

classifier = pipeline("sentiment-analysis", model="./blaze_nlp")
lstm_model = keras.models.load_model("lstm_model.h5")
scaler = pickle.load(open("scaler.pkl", "rb"))

2023-08-09 12:34:57.049412: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from typing import Union
import numpy as np


def derive_sentiment(model_output: {"label": str, "score": float}) -> float:
    """
    Derives the sentiment score from the model's output.

    :param model_output: A dictionary containing the model's classification label and score.
    :return: A sentiment score based on the model's output. Positive for 'yes_fire', negative for 'no_fire'.
    """

    return model_output["score"] * (1 if model_output["label"] == "yes_fire" else -1)


def predict_fire_from_temp(temperatures: Union[list[int], list[list[int]]]) -> list[dict]:
    """
    Predicts fire based on the given temperatures.

    :param temperatures: A list or list of lists containing temperature values.
    :return: A list of dictionaries containing the prediction labels ('no_fire' or 'yes_fire') and scores.
    """

    temperatures = np.array([temperatures]).reshape((-1, 7))
    temperatures = scaler.transform(temperatures)
    temperatures = temperatures.reshape(
        (temperatures.shape[0], temperatures.shape[1], 1)
    )

    # predict the fire
    predictions = lstm_model.predict(temperatures, verbose=0)
    output = []
    for prediction in list(predictions):
        prediction_result = np.argmax(prediction)
        label = ["no_fire", "yes_fire"][prediction_result]
        score = prediction[prediction_result]

        output.append({"label": label, "score": score})

    return output


def predict_fire(
    tweets: Union[list[str], list[list[str]]],
    temperatures: Union[list[int], list[list[int]]],
) -> list[float]:
    """
    Predicts fire based on the given tweets and temperatures.

    :param tweets: A list or list of lists containing tweets. Can be an empty list.
    :param temperatures: A list or list of lists containing temperature values.
    :return: A list of sentiment scores combining the information from tweets and temperatures.
             If tweets is an empty list, the function will return predictions based solely on temperatures.
    """

    if not isinstance(temperatures[0], list):
        temperatures = [temperatures]

    if not tweets:
        # Handle case when tweets is an empty list
        flat_temperatures = [temp for temp_batch in temperatures for temp in temp_batch]
        temperature_fire = predict_fire_from_temp(flat_temperatures)
        return [derive_sentiment(temp) for temp in temperature_fire]

    if len(tweets) and not isinstance(tweets[0], list):
        tweets = [tweets]

    # Flattening tweets and storing their batch indices
    flat_tweets = []
    tweet_batch_indices = [0]
    for tweet_batch in tweets:
        flat_tweets.extend(tweet_batch)
        tweet_batch_indices.append(len(flat_tweets))

    # Flattening temperatures and storing their batch indices
    flat_temperatures = [temp for temp_batch in temperatures for temp in temp_batch]
    temperature_batch_indices = [0]
    for i in range(len(temperatures)):
        temperature_batch_indices.append(
            temperature_batch_indices[-1] + len(temperatures[i])
        )

    # Get predictions for the flattened tweets and temperatures
    tweet_fire = classifier(flat_tweets)
    temperature_fire = predict_fire_from_temp(flat_temperatures)

    output = []

    # Process predictions based on indices
    for i in range(len(temperatures)):
        tweet_batch_start, tweet_batch_end = (
            tweet_batch_indices[i],
            tweet_batch_indices[i + 1],
        )
        tweet_batch_result = tweet_fire[tweet_batch_start:tweet_batch_end]
        temperature_batch_result = temperature_fire[i]

        average_tweet_sentiment = 0
        for tweet in tweet_batch_result:
            sentiment = derive_sentiment(tweet)
            average_tweet_sentiment += sentiment * (0.2 if sentiment < 0 else 1)
        average_tweet_sentiment = average_tweet_sentiment / len(tweet_batch_result)
        average_tweet_sentiment = round(average_tweet_sentiment)

        temperature_sentiment = derive_sentiment(temperature_batch_result)

        if not average_tweet_sentiment:
            output.append(temperature_sentiment)
        else:
            output.append(average_tweet_sentiment * 0.4 + temperature_sentiment * 0.6)

    return output

In [3]:
predict_fire(
    [
        ["i hate this town", "look at that forest fire go!", "wow that's crazy"],
        ["asdfsad", "asfdasdf"]
    ],
    [
        [155, 155, 155, 155, 155, 155, 155],
        [1, 2, 3, 4, 5, 6, 7]
    ],
)

[0.9999906301498414, -1.0]

In [4]:
# impossible case in real life, low temperature but tweets suggesting fire
predict_fire(
    ["i hate my life", "look at that forest fire go!", "wow that's crazy"],    
    [
        [1, 2, 3, 4, 5, 6, 7]
    ],
)

[-0.19999999999999996]

In [5]:
# sometimes tweets are unrelated to the fire, so the algorithm favors temperature data
predict_fire(
    ["asdfsad", "asfdasdf"],
    [
        [155, 155, 155, 155, 155, 155, 155]
    ],
)

[0.9999843835830688]

In [6]:
# Works even if there are no tweets that day
predict_fire(
    [],
    [
        [155, 155, 155, 155, 155, 155, 155]
    ],
)

[0.9999843835830688]